In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
import os
import shutil
import pickle
import cv2
import json
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Optimizer

In [2]:
file_path = 'C:/Users/Akoba/Desktop/START up/Covid19_research_project/data/raw/cough_sounds'
sound_metadata = pd.read_csv(file_path + '/sound-metadata.csv', encoding = 'latin1')

In [3]:
sound_metadata.columns

Index(['USER_ID', 'COUNTRY', 'AGE', 'COVID_STATUS', 'ENGLISH_PROFICIENCY',
       'GENDER', 'COUNTY_RO_STATE', 'CITY_LOCALITY', 'Diabetes', 'Asthma',
       'Smoker', 'Hypertension', 'Fever', 'Returning_User', 'Using_Mask',
       'Cold', 'Caugh', 'Muscle_Pain', 'loss_of_smell', 'Sore_Throat',
       'Fatigue', 'Breathing_Difficulties', 'Chronic_Lung_Disease',
       'Ischemic_Heart_Disease', 'Pneumonia', 'COVID_test_status', 'Diarrheoa',
       'DATES', 'breathing-deep', 'breathing-shallow', 'cough-heavy',
       'cough-shallow', 'counting-fast', 'counting-normal', 'vowel-a',
       'vowel-e', 'vowel-o'],
      dtype='object')

In [4]:
sound_metadata.drop('COUNTRY', axis=1, inplace=True)
sound_metadata.drop('GENDER', axis=1, inplace=True)
sound_metadata.drop('COUNTY_RO_STATE', axis=1, inplace=True)
sound_metadata.drop('CITY_LOCALITY', axis=1, inplace=True)
sound_metadata.drop('ENGLISH_PROFICIENCY', axis=1, inplace=True)
sound_metadata.drop('DATES', axis=1, inplace=True)

In [5]:
sound_metadata.columns

Index(['USER_ID', 'AGE', 'COVID_STATUS', 'Diabetes', 'Asthma', 'Smoker',
       'Hypertension', 'Fever', 'Returning_User', 'Using_Mask', 'Cold',
       'Caugh', 'Muscle_Pain', 'loss_of_smell', 'Sore_Throat', 'Fatigue',
       'Breathing_Difficulties', 'Chronic_Lung_Disease',
       'Ischemic_Heart_Disease', 'Pneumonia', 'COVID_test_status', 'Diarrheoa',
       'breathing-deep', 'breathing-shallow', 'cough-heavy', 'cough-shallow',
       'counting-fast', 'counting-normal', 'vowel-a', 'vowel-e', 'vowel-o'],
      dtype='object')

In [6]:
def label_for_cat(file_path):
    if 'train' in file_path:
        res = 'train'
    else:
        res = 'test'
    return res

In [7]:
sound_metadata['COVID_test_status'] = sound_metadata['COVID_test_status'].astype('str')
sound_metadata['COVID_LABEL'] = np.vectorize(label_for_cat)(sound_metadata['COVID_test_status'])

In [8]:
sound_metadata.columns
sound_metadata.count()

USER_ID                   1397
AGE                       1397
COVID_STATUS              1397
Diabetes                  1397
Asthma                    1397
Smoker                    1397
Hypertension              1397
Fever                     1397
Returning_User            1397
Using_Mask                1397
Cold                      1397
Caugh                     1397
Muscle_Pain               1397
loss_of_smell             1397
Sore_Throat               1397
Fatigue                   1397
Breathing_Difficulties    1397
Chronic_Lung_Disease      1397
Ischemic_Heart_Disease    1397
Pneumonia                 1397
COVID_test_status         1397
Diarrheoa                 1397
breathing-deep            1396
breathing-shallow         1396
cough-heavy               1396
cough-shallow             1395
counting-fast             1397
counting-normal           1397
vowel-a                   1396
vowel-e                   1396
vowel-o                   1395
COVID_LABEL               1397
dtype: i

In [9]:
# Fill NaN values in 'COVID_test_status' column with 1.0
sound_metadata['COVID_test_status'].fillna(1.0, inplace=True)

C:\Users\Akoba\AppData\Local\Temp\ipykernel_3128\1047930978.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sound_metadata['COVID_test_status'].fillna(1.0, inplace=True)


In [10]:
sound_metadata.count()

USER_ID                   1397
AGE                       1397
COVID_STATUS              1397
Diabetes                  1397
Asthma                    1397
Smoker                    1397
Hypertension              1397
Fever                     1397
Returning_User            1397
Using_Mask                1397
Cold                      1397
Caugh                     1397
Muscle_Pain               1397
loss_of_smell             1397
Sore_Throat               1397
Fatigue                   1397
Breathing_Difficulties    1397
Chronic_Lung_Disease      1397
Ischemic_Heart_Disease    1397
Pneumonia                 1397
COVID_test_status         1397
Diarrheoa                 1397
breathing-deep            1396
breathing-shallow         1396
cough-heavy               1396
cough-shallow             1395
counting-fast             1397
counting-normal           1397
vowel-a                   1396
vowel-e                   1396
vowel-o                   1395
COVID_LABEL               1397
dtype: i